ECE 447: From Transfer Functions to State Space
===

Prof. Eric Klavins

&copy; 2019, University of Washington

In [2]:
import numpy as np
import scipy.integrate as spi
import matplotlib.pyplot as plt
from sympy import *

%matplotlib inline

# Comment out in Google colab 
init_printing(use_latex='mathjax')

# Uncomment in below Google colab to render sympy equations nicely
# def custom_latex_printer(exp,**options):
#     from google.colab.output._publish import javascript
#     url = "https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.3/latest.js?config=default"
#     javascript(url=url)
#     return printing.latex(exp,**options)


# init_printing(use_latex="mathjax",latex_printer=custom_latex_printer)

Similar systems give the same transfer function
---

Suppose 

\begin{align}
\dot \x & = A \x + B u \\
y & = C x
\end{align}

and that

$$
{\bf z} = P^{-1} \x
$$

is a change of coordinates from $\x$ to ${\bf z}$.

Then

\begin{align}
\dot {\bf z} & = P^{-1} \dot \x \\
 & = P^{-1} ( A \x + B u ) \\
 & = P^{-1} ( A P{\bf z}  + B u ) \\
 & = P^{-1}A P {\bf z} + P^{-1} B u
\end{align}

and

$$
y = C \x = C P {\bf z} 
$$

so that the input/output dynamics of ${\bf z}$ are governed by the matrices 

$$
\tilde{A}=P^{-1}A P, \;\; \tilde{B}=P^{-1} B, \;\; \mathrm{and} \;\; \tilde{C}=C P 
$$

Now let's look at the transfer function in z-coordinates:

\begin{align}
sZ(s) & = P^{-1}AP Z(s) + P^{-1}BU(s) \\
s P Z(s) & = A P Z(s) + BU(s) \\
(sI-A)PZ(s) & = BU(s) \\
Z(s) & = P^{-1}(sI-A)^{-1}BU(s) \\
Y(s) & = CPZ(s) \\
     & = CPP^{-1}(sI-A)^{-1}BU(s) 
     & = C(sI-A)^{-1}BU(s)
\end{align}

so 

$$
\frac{Y(s)}{U(s)} = C(sI-A)^{-1}B
$$

which is the same as the transfer function derived from the original $\dot x$ system.

Since $P$ can be any invertible matrix, the above means that: 

> **A given transfer function has an infinite number of state space realizations.**

Which one you use depends on what you are trying to do. For now, we'll go over several ways to convert transfer functions into state space.

In this notebook, we'll come up with four different ways to turn a transfer function into state space. We'll use the following example throughout. 

Running Example
---

Consider a rocket with equations

$$
\begin{pmatrix}
\dot x \\
\dot v
\end{pmatrix} = \begin{pmatrix}
v \\
- \frac{g}{m} + f
\end{pmatrix}
$$

where $f$ is the thruster force. We are supposing the mass is not changing much during the time we will control it. You might assume we are trying to get the rocket to land on a landing pad at $x=0$ with $v=0$ so it doesn't hit too hard. Defining $u = -g/m + f$ we get the simple system

$$
\dot \x = \begin{pmatrix}
0 & 1 \\
0 & 0
\end{pmatrix} \x + \begin{pmatrix}
0 \\
1
\end{pmatrix} u
$$

If we suppose we have an altimeter, then $y = x$ and 

$$
y = ( 1 \;\; 0 ) \; \x
$$

The transfer function is 

\begin{align}
\frac{Y(s)}{U(s)} & = C ( sI - A ) ^{-1} B \\
                  & = ( 1 \;\; 0 ) \frac{1}{s^2} \begin{pmatrix} 
s & 1 \\
0 & s
\end{pmatrix} \begin{pmatrix}
0 \\
1
\end{pmatrix} \\
 & = \frac{1}{s^2}
\end{align}

Thus, the system is basically a double integrator. We can visualize it as a block diagram as follows:

<img width=30% src="https://raw.githubusercontent.com/klavins/ECE447/master/images/rocket.png">

If we suppose that we also have a sensor for the velocity, we can make a PI controller to control the velocity by using feedback. The controller has the transfer function $K_P + K_I/s$ which is the series composition of the proportional gain with the integrator, both taking the error $v_des - v$ as input.

<img width=40% src="https://raw.githubusercontent.com/klavins/ECE447/master/images/rocket-velocity.png">

Then we can control the whole system by supposing that the input to the above system, the desired velocity, is the output a PD controller that takes as input the desired position minus the actual position, to get:

<img width=55% src="https://raw.githubusercontent.com/klavins/ECE447/master/images/rocket-position.png">

The transfer function of this system can be found using the formula for a feedback interconnection we found last time. In particular, we show that the inner loop has transfer function

$$
L(s) = \frac{G(s)}{1+G(s)}
$$

where 

$$
G(s) = \frac{1}{s}\left(K_P + \frac{K_I}{s}\right)
$$

so that

$$
L(s) = \frac{K_Ps + K_I}{s^2 + K_Ps + K_I}.
$$

Then we can find the transfer function of the whole system using the same formula to get

$$
T(s) = \frac{H(s)}{1+H(s)}
$$

where 

$$
H(s) = \frac{1}{s}(K_{P_1} + K_Ds) L(s)
$$

Substituting in the above and simplifying we get

$$
T(s) = \frac{K_PK_Ds^2 + (K_DK_I + K_{P_1}K_P) s + K_{P_1}K_I}
            {s^3 + (K_P + K_PK_D)s^2 + (K_I+K_DK_I+K_PK_{P_1})s + K_IP_{P_1}}
$$

In [125]:
# In sympy
var("s G H L Kp Kp1 Kd Ki")
G = (Kp+Ki/s)/s
L = G / ( 1 + G )
H = (Kp1+Kd*s)*L/s
T = H / (1+H)
T = cancel(T) # Puts rational polynomials in a standard form
T

                           2                              
          Kd⋅Ki⋅s + Kd⋅Kp⋅s  + Ki⋅Kp₁ + Kp⋅Kp₁⋅s          
──────────────────────────────────────────────────────────
                 2                                  2    3
Kd⋅Ki⋅s + Kd⋅Kp⋅s  + Ki⋅Kp₁ + Ki⋅s + Kp⋅Kp₁⋅s + Kp⋅s  + s 

In [120]:
num,den = fraction(T)
num,den

⎛                 2                                       2                   
⎝Kd⋅Ki⋅s + Kd⋅Kp⋅s  + Ki⋅Kp₁ + Kp⋅Kp₁⋅s, Kd⋅Ki⋅s + Kd⋅Kp⋅s  + Ki⋅Kp₁ + Ki⋅s + 

               2    3⎞
Kp⋅Kp₁⋅s + Kp⋅s  + s ⎠

Phase Canonical Form
---

In *phase canonical form*, we convert a transfer function directly into a state space representation directly by using the coefficients of the numerator and denominator. Suppose that

$$
\frac{Y(s)}{U(s)} = \frac{b_3 s^3 + b_2 s^2 + b_1 s + b_0}
                         {s^4 + a_3 s^3 + a_2 s^2 + a_1 s + a_0}
$$

Make up a new signal called $Z(s)$ and multiply by $Z(s)/Z(s)$ to get 

\begin{align}
Y(s) & = (b_3 s^3 + b_2 s^2 + b_1 s + b_0) Z(s) \\
U(s) & = (s^4 + a_3 s^3 + a_2 s^2 + a_1 s + a_0) Z(s)
\end{align}

Then taking the inverse Laplace transform we get

\begin{align}
y & = b_3 \dddot z + b_2 \ddot z + b_1 \dot z + b_0 z \\
u & = \ddddot z + a_3 \dddot z + a_2 \ddot z + a_1 \dot z + a_0 z
\end{align}

If then define $x$ in terms of $z$ as follows:

\begin{align}
x_1 & = z \\
x_2 & = \dot z  = \dot x_1 \\
x_3 & = \ddot z = \dot x_2 \\
x_4 & = \dddot z = \dot x_3
\end{align}

then

\begin{align}
\dot x_4 & = \ddddot z = - a_3 \dddot z - a_2 \ddot z - a_1 \dot z - a_0 z + u \\
         & = -a_3 x_4 - a_2 x_3 - a_1 x_2 - a_0 x_1 + u
\end{align}

and 

$$
y = b_3 x_4 + b_2 x_3 + b_1 x_2 + b_0 x_1
$$

In matrix form then, we get 

\begin{align}
\dot x & = \begin{pmatrix}
0 & 1 & 0 & 0 \\
0 & 0 & 1 & 0 \\
0 & 0 & 0 & 1 \\
-a_0 & -a_1 & -a_2 & -a_3 
\end{pmatrix} \x + \begin{pmatrix}
0 \\
0 \\
0 \\
1
\end{pmatrix} u \\
y & = ( b_0 \; b_1 \; b_2 \; b_3 ) \; \x .
\end{align}

We can use sympy to get our running example into this form as follows. Note that the example system only has three states since the polynomial in the denominator has degree 3. What we get are $A$, $B$, and $C$ matrices that are equivalent to our original system, but which are not particularly intuitive. In particular, the output of the system is some combination of the state variables involving the controller gains.

In [99]:
# First we get the coefficients of the numerator of T
b = Poly(num,s).coeffs()
b

[Kd⋅Kp, Kd⋅Ki + Kp⋅Kp₁, Ki⋅Kp₁]

In [100]:
# Next we get the coefficients of the denominator
a = Poly(den,s).coeffs()[1:4]
a

[Kd⋅Kp + Kp, Kd⋅Ki + Ki + Kp⋅Kp₁, Ki⋅Kp₁]

In [101]:
# From these we can build the phase canonical form
A = Matrix([
    [0,1,0],
    [0,0,1],
]).row_insert(3,-Matrix([a]))
A
B = Matrix([[0],[0],[1]])
C = Matrix([b])
A,B,C

⎛⎡     0                1               0   ⎤, ⎡0⎤, [Kd⋅Kp  Kd⋅Ki + Kp⋅Kp₁  Ki
⎜⎢                                          ⎥  ⎢ ⎥                            
⎜⎢     0                0               1   ⎥  ⎢0⎥                            
⎜⎢                                          ⎥  ⎢ ⎥                            
⎝⎣-Kd⋅Kp - Kp  -Kd⋅Ki - Ki - Kp⋅Kp₁  -Ki⋅Kp₁⎦  ⎣1⎦                            

⋅Kp₁]⎞
     ⎟
     ⎟
     ⎟
     ⎠

In [102]:
# Check to see if we get the same transfer function back
T1 = (C*(s*eye(3)-A).inv()*B)[0]
T1 = cancel(T1)
num1,den1 = fraction(T1)
num1,den1

⎛                          2                                       2          
⎝Kd⋅Ki⋅s + Kd⋅Kp + Ki⋅Kp₁⋅s  + Kp⋅Kp₁⋅s, Kd⋅Ki⋅s + Kd⋅Kp + Ki⋅Kp₁⋅s  + Ki⋅s + 

                 3⎞
Kp⋅Kp₁⋅s + Kp + s ⎠

In [104]:
# They match the original 
num,den

⎛                 2                                       2                   
⎝Kd⋅Ki⋅s + Kd⋅Kp⋅s  + Ki⋅Kp₁ + Kp⋅Kp₁⋅s, Kd⋅Ki⋅s + Kd⋅Kp⋅s  + Ki⋅Kp₁ + Ki⋅s + 

               2    3⎞
Kp⋅Kp₁⋅s + Kp⋅s  + s ⎠

Controller Canonical Form
---

Phase canonical form is just one way of arranging the variables into matrices to get the same block diagram. Another way to do it is to notice the relationship between block diagrams and state space.

First, notice that an integrator block takes the derivative of a function $\dot x$ and integrates it to get $x$.

<img width=20% src="https://raw.githubusercontent.com/klavins/ECE447/master/images/integrator-alt.jpg">

A more complicated diagram can easily be turned into equations by following arrows and summations.

<img width=40% src="https://raw.githubusercontent.com/klavins/ECE447/master/images/controller-canonical-1d.jpg">

Here, we have 

\begin{align}
\dot x & = -a x + b u \\
y & = x
\end{align}

We can determine the transfer function of this system:

$$
T(s) = C(sI-A)^{-1}B = 1(s+a)^{-1}b = \frac{b}{s+a}
$$

Thus, this is in the form of equation (13) above.

In 2D, we get 

<img width=60% src="https://raw.githubusercontent.com/klavins/ECE447/master/images/controller-canonical-2d.jpg">

from which we can glean the equations

\begin{align}
\dot x_1 & = -a_1 x_1 + x_2 + b_1 u \\
\dot x_2 & = -a_0 x_1 + b_0 u \\
y & = x_1
\end{align}

which in matrix for is

\begin{align}
\dot \x & = \begin{pmatrix}
-a_1 & 1 \\
-a_0 & 0
\end{pmatrix} \x + \begin{pmatrix}
b_1 \\
b_0
\end{pmatrix} u \\
y & = (1\;0)\; \x
\end{align}

We can find the transfer function of this state space system as well, and see that it is still a variant of equation (13).

In [114]:
var("a0 a1 b0 b1 s")
A = Matrix([
    [-a1,1],
    [-a0,0]
])
B = Matrix([[b1],[b0]])
C = Matrix([[1, 0]])
T = (C*(s*eye(2)-A).inv()*B)[0]
T = cancel(T)
T

  b₀ + b₁⋅s   
──────────────
             2
a₀ + a₁⋅s + s 

Adding one more dimension should let you see the general structure of so-called controller canonical form.

<img width=80% src="https://raw.githubusercontent.com/klavins/ECE447/master/images/controller-canonical-3d.jpg">

Here, we get

\begin{align}
\dot \x & = \begin{pmatrix}
-a_2 & 1 & 0 \\
-a_1 & 0 & 1 \\
-a_0 & 0 & 0 
\end{pmatrix} \x + \begin{pmatrix}
b_2 \\
b_1 \\
b_0
\end{pmatrix} u \\
y & = (1\;0\;0)\; \x
\end{align}

and the transfer function is still a variant of equation (13):

In [117]:
var("a0 a1 a2 b0 b1 b2 s")
A = Matrix([
    [-a2,1,0],
    [-a1,0,1],
    [-a0,0,0]
])
B = Matrix([[b2],[b1],[b0]])
C = Matrix([[1,0,0]])
T = (C*(s*eye(3)-A).inv()*B)[0]
T = cancel(T)
T

                  2   
  b₀ + b₁⋅s + b₂⋅s    
──────────────────────
                2    3
a₀ + a₁⋅s + a₂⋅s  + s 

Physical Variable Form
---

The above suggests that we can read off a state space representation directly from the original block diagram, which is expanded a below to separate out the integrators in the two control blocks.


From the above we can see that 

\begin{align}
\dot x_1 & = x_2 \\
\dot x_2 & = K_p \dot x_3 + K_I x_3 \\
\dot x_3 & = K_{p_1}(u-x_1) - K_d x_2 - x_2
\end{align}

Substituting $\dot x_3$ into the right hand side of the equation for $\dot x_2$ gives

$$
\dot x_2 = K_p K_{p_1} u - K_p K_{p_1} x_1 - K_p K_d x_2 - K_p x_2 + K_I x_3
$$

Putting all of the above in matrix form yields:

\begin{align}
\dot \x & = \begin{pmatrix}
0 & 1 & 0 \\
-K_p K_{p_1} & -K_p K_d - K_p & K_I \\
-K_{p_1} & -K_d-1 & 0 
\end{pmatrix} \x + \begin{pmatrix}
0 \\
K_p K_{p_1} \\
0
\end{pmatrix} u \\
y & = (1\;0\;0)\; \x
\end{align}

We can once again check that this gives us our original transfer function.

In [128]:
var("a0 a1 a2 b0 b1 b2 s")
A = Matrix([
    [0,1,0],
    [-Kp*Kp1,-Kp*Kd-Kp,Ki],
    [-Kp1,-Kd-1,0]
])
B = Matrix([[0],[Kp*Kp1],[0]])
C = Matrix([[1,0,0]])
Tp = (C*(s*eye(3)-A).inv()*B)[0]
Tp = cancel(T)
Tp,T

⎛                           2                                                 
⎜          Kd⋅Ki⋅s + Kd⋅Kp⋅s  + Ki⋅Kp₁ + Kp⋅Kp₁⋅s                      Kd⋅Ki⋅s
⎜──────────────────────────────────────────────────────────, ─────────────────
⎜                 2                                  2    3                   
⎝Kd⋅Ki⋅s + Kd⋅Kp⋅s  + Ki⋅Kp₁ + Ki⋅s + Kp⋅Kp₁⋅s + Kp⋅s  + s   Kd⋅Ki⋅s + Kd⋅Kp⋅s

          2                              ⎞
 + Kd⋅Kp⋅s  + Ki⋅Kp₁ + Kp⋅Kp₁⋅s          ⎟
─────────────────────────────────────────⎟
2                                  2    3⎟
  + Ki⋅Kp₁ + Ki⋅s + Kp⋅Kp₁⋅s + Kp⋅s  + s ⎠

Diagonal Form
---

In [126]:
T

                           2                              
          Kd⋅Ki⋅s + Kd⋅Kp⋅s  + Ki⋅Kp₁ + Kp⋅Kp₁⋅s          
──────────────────────────────────────────────────────────
                 2                                  2    3
Kd⋅Ki⋅s + Kd⋅Kp⋅s  + Ki⋅Kp₁ + Ki⋅s + Kp⋅Kp₁⋅s + Kp⋅s  + s 